# 前提環境

公式: https://mamba.readthedocs.io/

Fabric のインストール

In [4]:
!pip install fabric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 555.3 kB/s eta 0:00:001m639.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 756.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 1.5 MB/s eta 0:00:001.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 2.2 MB/s eta 0:00:002.1 MB/s eta 0:00:01


## Fabric ローカル実行

In [32]:
from fabric import task
from invoke.context import Context
import shutil

@task
def CheckInstallation(c, command):
    result = shutil.which(command)
    print('=== fabfile.main.CheckInstallation()')
    print(f'--> shutil.which: {result}')
    
    return not result is None

CheckInstallation(Context(), 'rustup')

=== fabfile.main.CheckInstallation()
--> shutil.which: None


False

## Rust のインストール

In [1]:
!pwd

/home/tomoyan/my_projects/jupyter-binder/fabric_example


In [35]:
%%file ./fabfile/install_mamba.py
from fabric import task
from invoke.watchers import Responder

from . import CheckInstallation, tmux_run

@task
@tmux_run
def Mamba_CheckInstalled(c):
    result = CheckInstallation(c, 'rustup')
    return result

@task
@tmux_run
def Mamba_Install(c):
    if Mamba_CheckInstalled(c):
        print('Mamba はインストール済みです😉');
        return False
    
    enterResponder = Responder(
        pattern=r'\n>',
        response='\n'
    )
    result = c.run(
        # install.sh が shell を判定するので | bash である必要がある🤔
        #"curl --proto '=https' --tlsv1.2 -sSf https://micro.mamba.pm/install.sh | sh",
        "curl --proto '=https' --tlsv1.2 -sSf https://micro.mamba.pm/install.sh | bash",
        hide=False, pty=True,
        watchers=[enterResponder]
    )
    #print(result.stdout.strip())

@task
@tmux_run
def Mamba_Uninstall(c):
    if not Mamba_CheckInstalled(c):
        print('Mamba はインストールされていません🤔');
        return False
    
    yesResponder = Responder(
        pattern=r'Continue\? \(y/N\)',
        response='y\n'
    )
    result = c.run(
        "rustup self uninstall",
        pty=True,
        watchers=[yesResponder])


Overwriting ./fabfile/install_mamba.py


In [3]:
!fab --list

args: (<function Mamba_CheckInstalled at 0x7fac380f8d60>,), kwargs: {}
args: (<function Mamba_Install at 0x7fac37f43420>,), kwargs: {}
args: (<function Mamba_Uninstall at 0x7fac37f43560>,), kwargs: {}
args: (<function Rust_CheckInstalled at 0x7fac37f034c0>,), kwargs: {}
args: (<function Rust_Install at 0x7fac37f437e0>,), kwargs: {}
args: (<function Rust_Uninstall at 0x7fac37f43a60>,), kwargs: {}
args: (<function OLS_install at 0x7fac37f43d80>,), kwargs: {}
args: (<function OLS_uninstall at 0x7fac37f43ce0>,), kwargs: {}
Available tasks:

  CheckInstallation
  ConnectTmux
  Mamba-CheckInstalled
  Mamba-Install
  Mamba-Uninstall
  OLS-install
  OLS-uninstall
  Rust-CheckInstalled
  Rust-Install
  Rust-Uninstall



In [ ]:
!fab Mamba-CheckInstalled --debug

args: (<function Mamba_CheckInstalled at 0x7f92a7eece00>,), kwargs: {}
args: (<function Mamba_Install at 0x7f92a7d6b600>,), kwargs: {}
args: (<function Mamba_Uninstall at 0x7f92a7d6b740>,), kwargs: {}
args: (<function Rust_CheckInstalled at 0x7f92a7d37560>,), kwargs: {}
args: (<function Rust_Install at 0x7f92a7d6b9c0>,), kwargs: {}
args: (<function Rust_Uninstall at 0x7f92a7d6bc40>,), kwargs: {}
args: (<function OLS_install at 0x7f92a7d6bf60>,), kwargs: {}
args: (<function OLS_uninstall at 0x7f92a7d6bec0>,), kwargs: {}
7=[>c[>q[tomoyan@WICKED-BEAT python_example]$ ^C
[tomoyan@WICKED-BEAT python_example]$ 
[tomoyan@WICKED-BEAT python_example]$ ^C
[tomoyan@WICKED-BEAT python_example]$ 
[tomoyan@WICKED-BEAT python_example]$ ^C
[tomoyan@WICKED-BEAT python_example]$ 
[tomoyan@WICKED-BEAT python_example]$ ^C
[tomoyan@WICKED-BEAT python_example]$ 
[tomoyan@WICKED-BEAT python_example]$ 














[Fabric] 0:bash*                            [0,0] "WICKED-BEAT" 17:26 22- 4月-23[tomoyan@WI

In [ ]:
!fab Mamba-Install --debug

## Rust のアンインストール

In [38]:
!fab Rust-Uninstall

which: None
Rust はインストールされていません🤔


# 参考文献
https://docs.pyinvoke.org/en/stable/concepts/watchers.html<br>
https://blog.masu-mi.me/post/2021/02/14/catch-up-fabric2/<br>
https://docs.python.org/ja/3/library/inspect.html<br>
https://stackoverflow.com/questions/26515595/how-does-one-ignore-unexpected-keyword-arguments-passed-to-a-function<br>